<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-16 13:32:09
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.42 C
-------------------
Today PnL: -11.91 K (-0.08%)
Current PnL: -32.60 L (-20.38%)
CY Booked + Current PnL: -16.92 L (-10.58%)
-------------------
Total profit:  1.14 L
Total loss:  -33.74 L
-------------------
Total Booked + Current PnL: 10.06 L (7.56%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.05%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 97.97 L (69.07%)
Deployed:  1.33 C
Current:  1.42 C
CAGR/XIRR %: 3.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-2.67,0.06,21.49,21.57,21069.0,60.0,98040.0,397.04,-13.73,54.0,H-SC,3.92,167.0,0.00,0.72,43.07,XR,ATH,FINANCE
18,COALINDIA,3.24,3.82,14.83,19.23,23167.0,5754.0,156214.0,484.83,24.82,49.0,L-LC,11.37,185.0,0.25,1.14,23.85,XY25,ATH,MINING
77,TTKPRESTIG,-1.04,-27.47,37.99,0.09,27770.0,-27679.0,73098.0,770.00,69.69,38.0,M-SC,8.94,250.0,-1.00,0.53,0.60,OX40N,NTT,DURABLES
51,MEDANTA,0.55,-4.36,29.74,24.08,36108.0,-5538.0,121412.0,1486.00,-5.04,53.0,X-SC,11.64,87.0,-0.15,0.89,11.20,XY24,NTT,HEALTHCARE
36,ICICIGI,-0.22,1.28,17.84,19.34,36155.0,2552.0,202661.0,2252.93,-14.93,56.0,X-MC,3.16,66.0,0.07,1.48,17.42,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,14.37,-58.28,249.21,45.70,183842.0,-103045.0,73770.0,26.40,-23.41,65.0,M-SC,26.22,204.0,-0.56,0.54,21.54,XY24,NTT,TRAVEL
30,HAPPSTMNDS,3.85,-51.79,283.40,84.85,175119.0,-66373.0,61792.0,1480.71,-42.72,37.0,H-SC,25.42,152.0,-0.38,0.45,3.85,AR,ATH,IT
18,COALINDIA,3.24,3.82,14.83,19.23,23167.0,5754.0,156214.0,484.83,24.82,49.0,L-LC,11.37,185.0,0.25,1.14,23.85,XY25,ATH,MINING
48,KANSAINER,3.15,-31.14,64.78,13.47,120300.0,-83961.0,185706.0,340.00,-40.82,34.0,H-SC,8.57,162.0,-0.70,1.36,3.15,XY24,NTT,PAINTS
19,COFFEEDAY,2.83,-42.32,134.54,35.27,88112.0,-48058.0,65491.0,80.00,-55.73,58.0,L-SC,14.40,266.0,-0.55,0.48,60.29,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,-7.28,-48.46,237.90,74.15,200597.0,-79285.0,84320.0,838.0,-251.59,43.0,H-SC,24.53,145.0,-0.40,0.62,8.08,XR,NTT,CHEMICALS
5,ANGELONE,-5.33,4.61,18.73,24.20,59810.0,14075.0,319325.0,3033.0,51.87,47.0,X-SC,3.62,97.0,0.24,2.33,33.74,X40N,NTT,FINANCE
55,RAJESHEXPO,-4.56,-67.20,205.06,0.07,92271.0,-92180.0,44997.0,518.0,1548.99,45.0,L-SC,8.94,268.0,-1.00,0.33,11.91,OX40N,NTT,JEWELLERY
69,SURYODAY,-4.35,-29.25,73.90,23.03,93627.0,-52377.0,126694.0,216.0,38.16,29.0,H-SC,2.13,166.0,-0.56,0.93,25.71,XR,NTT,BANKS
66,SIS,-3.45,-27.41,67.06,21.26,54046.0,-30439.0,80593.0,528.0,1896.89,39.0,H-SC,4.09,168.0,-0.56,0.59,11.11,OX40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-2.67,0.06,21.49,21.57,21069.0,60.0,98040.0,397.04,-13.73,54.0,H-SC,3.92,167.0,0.0,0.72,43.07,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.53,2.15,103.54,107.91,156155.0,3172.0,150816.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.46,OX40N,ATH,CABLES
66,SIS,-3.45,-27.41,67.06,21.26,54046.0,-30439.0,80593.0,528.00,1896.89,39.0,H-SC,4.09,168.0,-0.56,0.59,11.11,OX40N,NTT,MISC
77,TTKPRESTIG,-1.04,-27.47,37.99,0.09,27770.0,-27679.0,73098.0,770.00,69.69,38.0,M-SC,8.94,250.0,-1.00,0.53,0.60,OX40N,NTT,DURABLES
40,INDIGOPNTS,1.40,-29.91,42.70,0.02,52244.0,-52208.0,122351.0,1408.00,86.38,32.0,M-SC,7.99,222.0,-1.00,0.89,6.42,OX40N,NTT,PAINTS
72,TATAELXSI,-0.89,-31.10,91.61,32.03,83218.0,-40996.0,90839.0,9161.00,-13.81,31.0,H-SC,6.40,158.0,-0.49,0.66,1.75,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,3.24,3.82,14.83,19.23,23167.0,5754.0,156214.0,484.83,24.82,49.0,L-LC,11.37,185.0,0.25,1.14,23.85,XY25,ATH,MINING
26,FINCABLES,-0.53,2.15,103.54,107.91,156155.0,3172.0,150816.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.46,OX40N,ATH,CABLES
50,MASFIN,-2.67,0.06,21.49,21.57,21069.0,60.0,98040.0,397.04,-13.73,54.0,H-SC,3.92,167.0,0.00,0.72,43.07,XR,ATH,FINANCE
1,ABB,1.93,8.10,34.66,45.57,98024.0,21197.0,282816.0,7934.00,-30.51,71.0,H-MC,2.88,125.0,0.22,2.07,25.59,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.93,8.10,34.66,45.57,98024.0,21197.0,282816.0,7934.00,-30.51,71.0,H-MC,2.88,125.0,0.22,2.07,25.59,AR,NTT,ELECTRICAL
26,FINCABLES,-0.53,2.15,103.54,107.91,156155.0,3172.0,150816.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.46,OX40N,ATH,CABLES
50,MASFIN,-2.67,0.06,21.49,21.57,21069.0,60.0,98040.0,397.04,-13.73,54.0,H-SC,3.92,167.0,0.00,0.72,43.07,XR,ATH,FINANCE
39,INDIAMART,0.16,-5.31,118.32,106.72,138181.0,-6550.0,116786.0,4810.62,-57.23,47.0,H-SC,7.65,140.0,-0.05,0.85,16.83,AR,ATH,MISC
85,ZYDUSLIFE,0.34,-4.27,41.68,35.63,83620.0,-8950.0,200624.0,1286.17,-16.67,52.0,H-MC,5.75,120.0,-0.11,1.47,11.69,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.75,-8.42,46.60,34.24,133522.0,-26360.0,286528.0,1972.00,-27.79,18.0,X-LC,4.53,3.0,-0.20,2.09,0.00,X40,NTT,IT
33,HCLTECH,-0.03,-5.59,31.17,23.83,71194.0,-13520.0,228404.0,1908.19,-0.74,22.0,X-LC,4.98,8.0,-0.19,1.67,9.58,X40,ATH,IT
73,TCS,-0.14,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,25.0,X-LC,3.06,2.0,-0.58,2.04,0.00,X40,ATH,IT
8,AWL,0.65,-34.24,139.05,57.19,298200.0,-111672.0,214455.0,485.00,-62.33,28.0,X-SC,17.28,81.0,-0.37,1.57,0.65,XY24,NTT,FMCG
13,BERGEPAINT,0.27,-17.47,46.79,21.14,87786.0,-39723.0,187616.0,680.00,-19.34,31.0,X-MC,11.57,68.0,-0.45,1.37,1.04,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.39,0.58,42.87,43.70,106964.0,1440.0,249508.0,735.00,-8.76,52.0,X-MC,1.80,70.0,0.01,1.82,15.72,XY24,BTT,FMCG
65,SIEMENS,2.12,-14.49,46.78,25.51,74441.0,-26965.0,159130.0,4671.50,33.57,57.0,H-LC,2.33,51.0,-0.36,1.16,17.27,AR,ATH,ELECTRICAL
78,UNITDSPR,-0.13,2.89,17.38,20.77,41869.0,6759.0,240903.0,1644.00,-8.78,58.0,X-MC,2.38,64.0,0.16,1.76,10.50,X40N,NTT,BREWERIES
81,VBL,1.76,-7.65,46.90,35.67,136804.0,-24148.0,291694.0,671.64,-18.49,44.0,X-LC,2.81,13.0,-0.18,2.13,5.15,X40N,ATH,FMCG
34,HINDUNILVR,0.60,-9.79,26.00,13.67,60294.0,-25163.0,231900.0,2922.00,-33.13,42.0,X-LC,2.84,9.0,-0.42,1.69,9.48,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-2.40,-46.34,151.98,35.20,104925.0,-59631.0,69039.0,2096.00,-9.46,34.0,X-SC,21.23,91.0,-0.57,0.50,0.00,X40,NTT,FOOTWEAR
42,INFY,-1.75,-8.42,46.60,34.24,133522.0,-26360.0,286528.0,1972.00,-27.79,18.0,X-LC,4.53,3.0,-0.20,2.09,0.00,X40,NTT,IT
58,RELAXO,-1.46,-54.08,231.69,52.31,154433.0,-78505.0,66655.0,1176.00,-51.12,33.0,X-SC,14.22,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
73,TCS,-0.14,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,25.0,X-LC,3.06,2.0,-0.58,2.04,0.00,X40,ATH,IT
2,ABBOTINDIA,0.04,-11.20,32.96,18.07,52347.0,-20035.0,158820.0,35195.00,-24.45,35.0,X-MC,10.30,58.0,-0.38,1.16,0.04,X40,ATH,PHARMA


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.94,-30.25,104.78,42.84,47511.0,-19662.0,45344.0,424.00,-54.27,51.0,X-SC,19.57,80.0,-0.41,0.33,9.07,XY24,NTT,MISC
58,RELAXO,-1.46,-54.08,231.69,52.31,154433.0,-78505.0,66655.0,1176.00,-51.12,33.0,X-SC,14.22,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.40,-46.34,151.98,35.20,104925.0,-59631.0,69039.0,2096.00,-9.46,34.0,X-SC,21.23,91.0,-0.57,0.50,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,0.55,-4.36,29.74,24.08,36108.0,-5538.0,121412.0,1486.00,-5.04,53.0,X-SC,11.64,87.0,-0.15,0.89,11.20,XY24,NTT,HEALTHCARE
35,HONAUT,-0.57,-23.39,87.07,43.32,108646.0,-38092.0,124780.0,58357.33,-32.74,37.0,X-SC,10.63,90.0,-0.35,0.91,1.32,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.29,-22.11,42.23,10.78,79048.0,-53134.0,187184.0,452.00,-53.90,42.0,X-LC,19.02,1.0,-0.67,1.37,3.10,X40,NTT,FMCG
73,TCS,-0.14,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,25.0,X-LC,3.06,2.0,-0.58,2.04,0.00,X40,ATH,IT
42,INFY,-1.75,-8.42,46.60,34.24,133522.0,-26360.0,286528.0,1972.00,-27.79,18.0,X-LC,4.53,3.0,-0.20,2.09,0.00,X40,NTT,IT
75,TMPV,-0.89,-22.76,59.21,22.98,103311.0,-51416.0,174482.0,600.00,-82.53,60.0,X-LC,3.25,4.0,-0.50,1.27,11.54,XY24,NTT,AUTO
33,HCLTECH,-0.03,-5.59,31.17,23.83,71194.0,-13520.0,228404.0,1908.19,-0.74,22.0,X-LC,4.98,8.0,-0.19,1.67,9.58,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.11,-31.56,94.16,32.89,49756.0,-24364.0,52842.0,1800.00,-348.60,59.0,L-MC,11.03,258.0,-0.49,0.39,45.52,XR,NTT,BANKS
67,SONACOMS,-0.86,-9.45,53.61,39.09,49104.0,-9564.0,91595.0,804.02,-28.30,60.0,M-MC,5.12,193.0,-0.19,0.67,29.58,AR,ATH,AUTO
50,MASFIN,-2.67,0.06,21.49,21.57,21069.0,60.0,98040.0,397.04,-13.73,54.0,H-SC,3.92,167.0,0.00,0.72,43.07,XR,ATH,FINANCE
79,VAIBHAVGBL,-2.35,-23.70,110.69,60.77,154375.0,-43309.0,139466.0,521.00,60.88,56.0,H-SC,0.80,160.0,-0.28,1.02,26.44,XR,NTT,JEWELLERY
18,COALINDIA,3.24,3.82,14.83,19.23,23167.0,5754.0,156214.0,484.83,24.82,49.0,L-LC,11.37,185.0,0.25,1.14,23.85,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.93,8.10,34.66,45.57,98024.0,21197.0,282816.0,7934.00,-30.51,71.0,H-MC,2.88,125.0,0.22,2.07,25.59,AR,NTT,ELECTRICAL
7,ATULAUTO,0.59,-13.19,67.69,45.57,115832.0,-26010.0,171122.0,844.00,3710.81,67.0,M-SC,5.08,248.0,-0.22,1.25,29.50,XY24,NTT,AUTO
79,VAIBHAVGBL,-2.35,-23.70,110.69,60.77,154375.0,-43309.0,139466.0,521.00,60.88,56.0,H-SC,0.80,160.0,-0.28,1.02,26.44,XR,NTT,JEWELLERY
11,BANDHANBNK,1.02,-19.61,139.35,92.42,312062.0,-54618.0,223941.0,400.00,91.81,67.0,H-SC,7.84,174.0,-0.18,1.64,29.42,XY24,NTT,BANKS
67,SONACOMS,-0.86,-9.45,53.61,39.09,49104.0,-9564.0,91595.0,804.02,-28.30,60.0,M-MC,5.12,193.0,-0.19,0.67,29.58,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.21
1,25,44.61
2,50,76.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.25
MC    30.48
LC    23.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.77
X40      23.13
X40N     15.25
AR        9.05
XY25      8.79
XR        8.75
OX40N     7.34
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.01
H-SC    22.68
X-LC    19.85
X-SC    11.85
M-SC    10.91
H-MC     5.00
M-MC     2.08
H-LC     1.16
L-LC     1.14
M-LC     1.07
L-SC     0.81
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.02,-15.07,53.74
FINANCE,13.34,-14.38,59.38
IT,11.50,-35.57,103.22
MISC,7.12,-33.19,87.67
ELECTRICAL,6.32,-8.41,47.97
PAINTS,5.16,-30.71,50.13
INSURANCE,4.86,-1.88,36.75
PHARMA,4.15,-3.46,36.02
AUTO,3.19,-19.90,61.36


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3309390.0,21
AR,1331919.0,10
X40,1327091.0,15
XR,1295352.0,12
X40N,1033167.0,10
OX40N,785137.0,10
XY25,415114.0,6
SR,299525.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3686635.0,24
M-SC,1423675.0,14
X-MC,1360702.0,15
X-SC,1181977.0,10
X-LC,1172699.0,12
H-MC,395960.0,3
L-SC,180383.0,2
M-LC,145758.0,1
M-MC,101542.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253665.0      6
           AR         951784.0      5
           XR         831986.0      7
M-SC       XY24       825798.0      6
X-SC       XY24       608552.0      4
X-LC       X40        602942.0      6
X-MC       X40        544748.0      7
           X40N       410397.0      4
X-SC       X40N       394024.0      4
H-SC       OX40N      349675.0      4
M-SC       OX40N      343191.0      5
H-SC       SR         299525.0      2
X-LC       X40N       228746.0      2
H-MC       XY24       214316.0      1
X-LC       XY24       212309.0      2
X-MC       XY25       210807.0      2
           XY24       194750.0      2
H-MC       AR         181644.0      2
M-SC       XR         179740.0      2
X-SC       X40        179401.0      2
M-LC       XR         145758.0      1
X-LC       XY25       128702.0      2
L-SC       OX40N       92271.0      1
           XR          88112.0      1
M-SC       AR          74946.0      1
H-LC       AR          74441.0      1
M-MC       XY25        52438.0      1
L-MC       XR          49756.0      1
M-MC       AR          49104.0      1
L-LC       XY25        23167.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
